In [ ]:
#to do: it's unnecessary to open each file and copy each var / dimension manually like this. 
#       simplify - open existing file and only change the vars we want to change. 
#       in the open file function it will need to call another open file function to 
#       get values so they can be shifted slightly
#       - double check times for each files from 1990 on to make sure I didn't screw up
#       (use df.DataFrame(time).tail())
#   

In [ ]:
# ERA5 atmospheric forcing data (daily, hourly) needs adjustment
# - generate relative humidity from 2 m dewpoint temp
# - u10,v10,t2,dewpoint2 etc are snapshots on the hour.
#   the means are over the preceeding hour, so the record for 01:00 
#   really corresponds to 00:30

# log
#  2021-05-22 - created script
#  
# in: 
#   1) daily ERA5 model files with 9 variables, hourly
#   2)
#
# out: 
#   1) 

In [2]:
#GO's changelog: 
#   - added year_start year_end variable
#   - redownloaded data and changed rad flux vars to:  msdwlwrf, msdwswrf
#   - lines 28-31 - may need changes depending on local env (Metpy 1.0 vs MetPy 0.12)
import os
import shutil
import metpy.calc as mpcalc
import netCDF4 as nc
import numpy as np
from metpy.units import units


# def getrh(t2m, d2m):
#     #################################################################################################
#     # relative humidity calculation
#     # https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.relative_humidity_from_dewpoint.html
#     # use the 'pint' library bundled with metpy to declare units
#     tair_all_K = t2m.filled() * units.degK
#     d2m_all_K = d2m.filled() * units.degK
#     rh = mpcalc.relative_humidity_from_dewpoint(tair_all_K, d2m_all_K).magnitude
#     rh = np.clip(rh, 0, 1)
#     return rh

def getq(d2m,pressure):
    # specific humidity calculation
    pressurePa = pressure.filled() * units.Pa  # assume pressure at surface is good enough for pressure at 2m
    d2mK = d2m.filled() * units.degK
    #metpy 1.0
    #q2m = mpcalc.specific_humidity_from_dewpoint(pressurePa, d2mK).magnitude
    #metpy 0.12
    q2m = mpcalc.specific_humidity_from_dewpoint(d2mK, pressurePa).magnitude
    return q2m


def get_shifted_data(var, year):
    infile1 = os.path.join(inpath, 'ERA5_SalishSea_9vars_{}.nc'.format(str(year)))
    infile2 = os.path.join(inpath, 'ERA5_SalishSea_9vars_{}.nc'.format(str(year+1)))
    with nc.Dataset(infile1) as ncf:
        tmp = np.zeros(ncf[var].shape) - 999
        tmp[:-1,:,:] = ncf[var][1:,:,:]  # second record becomes first, etc
    if year == year_end:
        # we don't have a 2021 file, duplicate last record as a workaround
        tmp[-1, :, :] = tmp[-2, :, :]
    else:
        with nc.Dataset(infile2) as ncf:
            tmp[-1,:,:] = ncf[var][0,:,:]  # first record from next year is last record of this year
    return tmp


#inpath='/home/mid002/WORK4/SalishSea1500/ECMWF_ERA5/original/'
#outpath='/home/mid002/WORK4/SalishSea1500/ECMWF_ERA5/adjusted_md/'
inpath='C://Users//Greig//Documents//GitHub//NEMO-Salish-Sea-2021//code//'
outpath='C://Users//Greig//Documents//GitHub//NEMO-Salish-Sea-2021//data//forcing/ECMWF//ERA5//'
os.makedirs(outpath,exist_ok=True)
year_start = 1979
year_end = 2015

for year in range(year_start, year_end+1):
    print(year)
    infile = os.path.join(inpath, 'ERA5_SalishSea_9vars_{}.nc'.format(year))
    outfile = os.path.join(outpath, 'ERA5_SalishSea_fixed_{}.nc'.format(year))

    shutil.copy2(infile,outfile)  # copy original file

    # load shifted data
    msr = get_shifted_data('msr', year)
    mtpr = get_shifted_data('mtpr', year)
    msdwlwrf = get_shifted_data('msdwlwrf', year)  
    msdwswrf = get_shifted_data('msdwswrf', year)  

    with nc.Dataset(outfile,'r+') as ncf:
        # add relative humidity variable
        q = ncf.createVariable('q2m', "float32", ("time", "latitude", "longitude"))
        q.units = "dimensionless"
        q.long_name = "specific_humidity"
        q.missing_value = -999
        q.scale_factor = 1
        q.add_offset = 0

        # compute and store q
        q[:,:,:] = getq(ncf['d2m'][:, :, :], ncf['sp'][:,:,:])

        # write shifted data
        ncf['msr'][:,:,:] = msr
        ncf['mtpr'][:, :, :] = mtpr
        ncf['msdwlwrf'][:, :, :] = msdwlwrf
        ncf['msdwswrf'][:, :, :] = msdwswrf

1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015


## Old code

In [2]:
#%matplotlib notebook
import netCDF4 as nc
import numpy as np
import metpy
import metpy.calc as mpcalc
from metpy.units import units
import os

start_year = 1979
end_year = 2015
year = start_year

# load ERA5 - 9 vars, 2D+time
# name format: ERA5_SalishSea_9vars_1979.nc
def load_era5(f):
    with nc.Dataset(f) as ncid:
        # to do - can replace most of below with loop over vars
        latt = ncid.variables["latitude"][:].filled()
        lont = ncid.variables["longitude"][:].filled()
        
        time = ncid.variables["time"][:].filled()
        time_units = ncid.variables["time"].units
        # :units = "hours since 1900-01-01 00:00:00.0";
        # :calendar = "gregorian";
        
        tair = ncid.variables["t2m"][:, :, :].filled() # 2 m dewpoint temp
        tair_units = ncid.variables["t2m"].units # Kelvin!
        tair_scale_factor = ncid.variables["t2m"].scale_factor
        tair_add_offset = ncid.variables["t2m"].add_offset
        tair_fill_value = ncid.variables["t2m"]._FillValue
        tair_missing_value = ncid.variables["t2m"].missing_value
        
        d2m = ncid.variables["d2m"][:, :, :].filled() # 2 m dewpoint temp
        d2m_units = ncid.variables["d2m"].units 
        d2m_scale_factor = ncid.variables["d2m"].scale_factor
        d2m_add_offset = ncid.variables["d2m"].add_offset
        d2m_fill_value = ncid.variables["d2m"]._FillValue
        d2m_missing_value = ncid.variables["d2m"].missing_value
        
        qlw = ncid.variables["msnlwrfcs"][:, :, :].filled() # mean surface net long-wave rad flux
        qlw_units = ncid.variables["msnlwrfcs"].units
        qlw_scale_factor = ncid.variables["msnlwrfcs"].scale_factor
        qlw_add_offset = ncid.variables["msnlwrfcs"].add_offset
        qlw_fill_value = ncid.variables["msnlwrfcs"]._FillValue
        qlw_missing_value = ncid.variables["msnlwrfcs"].missing_value
        
        qsr = ncid.variables["msnswrfcs"][:, :, :].filled() # mean surface net short-wave rad flux
        qsr_units = ncid.variables["msnswrfcs"].units
        qsr_scale_factor = ncid.variables["msnswrfcs"].scale_factor
        qsr_add_offset = ncid.variables["msnswrfcs"].add_offset
        qsr_fill_value = ncid.variables["msnswrfcs"]._FillValue
        qsr_missing_value = ncid.variables["msnswrfcs"].missing_value
        
        snow = ncid.variables["msr"][:, :, :].filled() # mean snowfall rate
        snow_units = ncid.variables["msr"].units
        snow_scale_factor = ncid.variables["msr"].scale_factor
        snow_add_offset = ncid.variables["msr"].add_offset
        snow_fill_value = ncid.variables["msr"]._FillValue
        snow_missing_value = ncid.variables["msr"].missing_value
        
        prec = ncid.variables["mtpr"][:, :, :].filled() # mean total precip rate
        prec_units = ncid.variables["mtpr"].units
        prec_scale_factor = ncid.variables["mtpr"].scale_factor
        prec_add_offset = ncid.variables["mtpr"].add_offset
        prec_fill_value = ncid.variables["mtpr"]._FillValue
        prec_missing_value = ncid.variables["mtpr"].missing_value
        
        sp = ncid.variables["sp"][:, :, :].filled() # surface pressure
        sp_units = ncid.variables["sp"].units
        sp_scale_factor = ncid.variables["sp"].scale_factor
        sp_add_offset = ncid.variables["sp"].add_offset
        sp_fill_value = ncid.variables["sp"]._FillValue
        sp_missing_value = ncid.variables["sp"].missing_value
        
        windi = ncid.variables["u10"][:, :, :].filled() # 10m u-component of wind
        windi_units = ncid.variables["u10"].units
        windi_scale_factor = ncid.variables["u10"].scale_factor
        windi_add_offset = ncid.variables["u10"].add_offset
        windi_fill_value = ncid.variables["u10"]._FillValue
        windi_missing_value = ncid.variables["u10"].missing_value
        
        windj = ncid.variables["v10"][:, :, :].filled() # 10m v-component of wind
        windj_units = ncid.variables["v10"].units
        windj_scale_factor = ncid.variables["v10"].scale_factor
        windj_add_offset = ncid.variables["v10"].add_offset
        windj_fill_value = ncid.variables["v10"]._FillValue
        windj_missing_value = ncid.variables["v10"].missing_value
        
        relhum_units = "dimensionless"
        relhum_scale_factor = 1
        relhum_add_offset = 0
        relhum_fill_value = tair_fill_value
        relhum_missing_value = tair_missing_value
        
        var_values = [tair, d2m, qlw, qsr, snow, prec, sp, windi, windj]
        var_missing_values = [tair_missing_value,d2m_missing_value,qlw_missing_value,
                              qsr_missing_value,snow_missing_value,prec_missing_value,
                              sp_missing_value,windi_missing_value,windj_missing_value,
                              relhum_missing_value]
        var_fill_values = [tair_fill_value, d2m_fill_value, qlw_fill_value,
                           qsr_fill_value, snow_fill_value, prec_fill_value,
                           sp_fill_value, windi_fill_value, windj_fill_value,
                           relhum_fill_value]
        var_add_offsets = [tair_add_offset, d2m_add_offset, qlw_add_offset,
                           qsr_add_offset, snow_add_offset, prec_add_offset,
                           sp_add_offset, windi_add_offset, windj_add_offset,
                           relhum_add_offset]
        var_scale_factors = [tair_scale_factor, d2m_scale_factor, qlw_scale_factor,
                             qsr_scale_factor, snow_scale_factor, prec_scale_factor,
                             sp_scale_factor, windi_scale_factor, windj_scale_factor,
                             relhum_scale_factor]
        var_units = [tair_units, d2m_units, qlw_units,
                     qsr_units, snow_units, prec_units,
                     sp_units, windi_units, windj_units,
                     relhum_units]
        
    return (latt, lont, time, time_units, 
            var_values, var_missing_values, 
            var_fill_values, var_add_offsets,
            var_scale_factors, var_units)


def write_netcdf(fname, time, time_units, length_year, latt, lont, 
                 var_values, var_missing_values,
                 var_fill_values, var_add_offsets,
                 var_scale_factors, var_units):
    
    """ Write specified variables to nc file. """
    # use variable names close to what NEMO expects
    var_names = ['tair', 'd2m','qlw','qsr','snow','precip','sp','u10','v10','relhum']
    var_longnames = ['temp at 2 metres', 
                     '2 metre dewpoint temperature',
                     'Mean surface net long-wave radiation flux, clear sky',
                     'Mean surface net short-wave radiation flux, clear sky',
                     'Mean snowfall rate',
                     'Mean total precipitation rate',
                     'Surface pressure',
                     '10 metre U wind component',
                     '10 metre V wind component',
                     'relative humidity at 2 m']
    ny, nx = (len(latt), len(lont))
    
    # append if file exists, otherwise write a new file
    with nc.Dataset(fname, 'a' if os.path.isfile(fname) else 'w') as ncf:
        
        # dimensions
        if 'time' not in ncf.dimensions:
            ncf.createDimension("time", None)

        if 'time' not in ncf.variables:
            time_counter = ncf.createVariable("time", "double", ("time"), zlib=True, fill_value=0.0)
            time_counter.standard_name = "time"
            time_counter.units = time_units
            time_counter.calendar = "gregorian"
            ncf.variables['time'][0:length_year] = time[0:length_year]
        else:
            time_counter = ncf.variables['time']
        
        if 'latt' not in ncf.dimensions:
            ncf.createDimension("latt", ny)
        if 'latt' not in ncf.variables:
            latt_1 = ncf.createVariable("latt", "double", "latt", zlib=True, fill_value=0.0)
            latt_1.standard_name = "latitude"
            latt_1.units = "degrees_north"
            latt_1[:] = latt
        else:
            latt_1 = ncf.variables['latt']

        if 'lont' not in ncf.dimensions:
            ncf.createDimension("lont", nx)
        if 'lont' not in ncf.variables:
            lont_1 = ncf.createVariable("lont", "double", "lont", zlib=True, fill_value=0.0)
            lont_1.standard_name = "longitude"
            lont_1.units = "degress_east"
            lont_1[:] = lont
        else:
            lont_1 = ncf.variables['lont']

        # variables
        for vname, units, long_names, vals, missing, fill, scale, offset in zip(var_names, var_units, 
                                                                                var_longnames, var_values, 
                                                                                var_missing_values, var_fill_values,
                                                                                var_scale_factors, var_add_offsets):
            if vals is not None:
                if vname not in ncf.variables:
                    var = ncf.createVariable(vname, "float32", ("time","latt","lont"), zlib=True, fill_value=fill)
                    var.units = units
                    var.long_name = long_names
                    var.missing_value = missing
                    var.scale_factor = scale
                    var.add_offset = offset
                if vname in {'qlw', 'qsr', 'snow', 'precip'}:
                    ncf.variables[vname][0:length_year,:,:] = vals[1:length_year+1,:,:]
                else:
                    ncf.variables[vname][0:length_year,:,:] = vals[0:length_year,:,:]

#################################################################################################
# outer loop for years
# inner loop for year + next year                    
                    
outerloop_year = start_year
for outerloop_year in range(start_year, end_year + 1, 1):
    
    #################################################################################################
    # get values from this year and one year ahead
    for year in range(outerloop_year, outerloop_year + 2, 1):
        f = "ERA5_SalishSea_9vars_" + str(year) + ".nc"
        
        latt, lont, time2, time_units, var_values, var_missing_values, var_fill_values, var_add_offsets, var_scale_factors, var_units = load_era5(f)
       
        if year == outerloop_year:

            time_all = time2
            length_year = len(time2)
            tair_all = var_values[0]
            d2m_all = var_values[1]
            qlw_all = var_values[2]
            qsr_all = var_values[3]
            snow_all = var_values[4]
            prec_all = var_values[5]
            sp_all = var_values[6]
            windi_all = var_values[7]
            windj_all = var_values[8]
        else:
            time_all = np.append(time_all, time2)
            tair_all = np.append(tair_all, var_values[0], axis=0)
            d2m_all = np.append(d2m_all, var_values[1], axis=0)
            qlw_all = np.append(qlw_all, var_values[2], axis=0)
            qsr_all = np.append(qsr_all, var_values[3], axis=0)
            snow_all = np.append(snow_all, var_values[4], axis=0)
            prec_all = np.append(prec_all, var_values[5], axis=0)
            sp_all = np.append(sp_all, var_values[6], axis=0)
            windi_all = np.append(windi_all, var_values[7], axis=0)
            windj_all = np.append(windj_all, var_values[8], axis=0)

    
    #################################################################################################
    # relative humidity calculation
    # https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.relative_humidity_from_dewpoint.html
    # use the 'pint' library bundled with metpy to declare units

    tair_all_K = tair_all * units.degK
    d2m_all_K = d2m_all * units.degK
    relhum_all = mpcalc.relative_humidity_from_dewpoint(tair_all_K, d2m_all_K)
    relhum2 = relhum_all[:,:,:].magnitude
    
    var_values2 = []
    var_values2 = [tair_all, d2m_all, qlw_all, qsr_all, snow_all, prec_all, sp_all, windi_all, windj_all,relhum2]

    #################################################################################################
    # write to file
    fname = "ERA5_SalishSea_fixed_" + str(year-1) + ".nc"
    print(year-1)
    print(length_year)
    print(tair_all.shape)
    write_netcdf(fname, time_all, time_units, length_year, latt, lont, var_values2, var_missing_values, var_fill_values, var_add_offsets, var_scale_factors, var_units)


2019
8760
(17544, 18, 26)


In [300]:
for year in range(outerloop_year, outerloop_year + 2, 1):
    print(year)

1994
1995


In [281]:
len(tair_all)

52584

In [243]:
(relhum_all.magnitude).shape

(175296, 18, 26)

In [302]:
tair_all.shape

(17520, 18, 26)

In [283]:
var_values[1].shape

(8760, 18, 26)

In [293]:
tair_all = var_values[0]
tair_all = np.append(tair_all, var_values[0], axis=0)
tair_all.shape

(17520, 18, 26)

In [294]:
time.shape

(8760,)

In [240]:
year

1990

In [12]:
pip freeze

affine==2.3.0
aiohttp==3.7.2
alabaster==0.7.12
altair==3.2.0
anaconda-client==1.7.2
anaconda-navigator==1.9.12
anaconda-project==0.8.3
angles==2.0
appdirs==1.4.4
arrow==0.17.0
asgiref==3.2.10
asn1crypto==1.0.1
astroid==2.3.1
astropy==3.2.1
async-timeout==3.0.1
atomicwrites==1.3.0
attrs==19.2.0
Babel==2.7.0
backcall==0.1.0
backports.functools-lru-cache==1.6.4
backports.os==0.1.1
backports.shutil-get-terminal-size==1.0.0
backports.tempfile==1.0
backports.weakref==1.0.post1
basemap==1.2.0
beautifulsoup4==4.8.0
bitarray==1.0.1
bkcharts==0.2
bleach==3.1.0
bokeh==1.3.4
boto==2.49.0
Bottleneck==1.2.1
Brotli==1.0.9
Cartopy==0.17.0
cdsapi==0.5.1
celluloid==0.2.0
certifi==2019.9.11
cffi==1.12.3
cftime==1.0.4.2
chardet==3.0.4
chart-studio==1.1.0
Click==7.0
click-plugins==1.1.1
cligj==0.7.1
cloudpickle==1.2.2
clyent==1.2.2
cmocean==2.0
colorama==0.4.1
comtypes==1.1.7
conda==4.10.1
conda-build==3.18.9
conda-package-handling==1.7.3
conda-verify==3.4.2
contextily==1.1.0
contextlib2==0.6.0
cryptograph